In [6]:
links = [
    'https://ogmmateryal.eba.gov.tr/soru-bankasi/tde',
    'https://ogmmateryal.eba.gov.tr/soru-bankasi/tarih',
    'https://ogmmateryal.eba.gov.tr/soru-bankasi/cografya', 
    'https://ogmmateryal.eba.gov.tr/soru-bankasi/felsefe',
    'https://ogmmateryal.eba.gov.tr/soru-bankasi/inkilap-tarihi',
    'https://ogmmateryal.eba.gov.tr/soru-bankasi/matematik',
    'https://ogmmateryal.eba.gov.tr/soru-bankasi/fizik',
    'https://ogmmateryal.eba.gov.tr/soru-bankasi/kimya', 
    'https://ogmmateryal.eba.gov.tr/soru-bankasi/biyoloji',
    'https://ogmmateryal.eba.gov.tr/soru-bankasi/ingilizce',
    'https://ogmmateryal.eba.gov.tr/soru-bankasi/fl-matematik',
    'https://ogmmateryal.eba.gov.tr/soru-bankasi/fl-fizik',
    'https://ogmmateryal.eba.gov.tr/soru-bankasi/fl-kimya',
    'https://ogmmateryal.eba.gov.tr/soru-bankasi/fl-biyoloji',
    'https://ogmmateryal.eba.gov.tr/soru-bankasi/dikab'
]


import requests
from bs4 import BeautifulSoup

lessons = {}

for link in links:
    # Get the page content
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all elements with class "books-detail-box"
    detail_boxes = soup.find_all(class_='books-detail-box')

    # Extract links from each detail box
    lesson_links = []
    for box in detail_boxes:
        links_in_box = box.find_all('a')
        for a in links_in_box:
            href = a.get('href')
            if href:
                lesson_links.append({
                    'text': a.get_text(strip=True),
                    'href': href
                })
    
    # Store links for this lesson
    lesson_name = link.split('/')[-1]
    lessons[lesson_name] = lesson_links

print(f"Found links for {len(lessons)} lessons")
for lesson, links in lessons.items():
    print(f"{lesson}: {links} links")


Found links for 15 lessons
tde: [{'text': '10. Sınıf', 'href': '/soru-bankasi-kazanim/tde?s=7&d=59&u=0&k=0'}, {'text': '11. Sınıf', 'href': '/soru-bankasi-kazanim/tde?s=8&d=60&u=0&k=0'}, {'text': '12. Sınıf', 'href': '/soru-bankasi-kazanim/tde?s=9&d=61&u=0&k=0'}, {'text': '9. Sınıf (2017-23 Müfredatı)', 'href': '/soru-bankasi-kazanim/tde?s=6&d=58&u=0&k=0'}] links
tarih: [{'text': '10. Sınıf', 'href': '/soru-bankasi-kazanim/tarih?s=7&d=56&u=0&k=0'}, {'text': '11. Sınıf', 'href': '/soru-bankasi-kazanim/tarih?s=8&d=57&u=0&k=0'}, {'text': '9. Sınıf (2017-23 Müfredatı)', 'href': '/soru-bankasi-kazanim/tarih?s=6&d=55&u=0&k=0'}] links
cografya: [{'text': '10. Sınıf', 'href': '/soru-bankasi-kazanim/cografya?s=7&d=10&u=0&k=0'}, {'text': '11. Sınıf', 'href': '/soru-bankasi-kazanim/cografya?s=8&d=11&u=0&k=0'}, {'text': '12. Sınıf', 'href': '/soru-bankasi-kazanim/cografya?s=9&d=12&u=0&k=0'}, {'text': '9. Sınıf (2017-23 Müfredatı)', 'href': '/soru-bankasi-kazanim/cografya?s=6&d=9&u=0&k=0'}] links
f

In [10]:
# Add base URL to all lesson links
base_url = "https://ogmmateryal.eba.gov.tr"
for lesson in lessons:
    for link in lessons[lesson]:
        link['href'] = base_url + link['href']

# Get first lesson name and links to verify
first_lesson = next(iter(lessons))
print(f"\nFirst lesson: {first_lesson}")
print("\nLinks:")
for link in lessons[first_lesson]:
    print(f"Text: {link['text']}")
    print(f"URL: {link['href']}")



First lesson: tde

Links:
Text: 10. Sınıf
URL: https://ogmmateryal.eba.gov.tr/soru-bankasi-kazanim/tde?s=7&d=59&u=0&k=0
Text: 11. Sınıf
URL: https://ogmmateryal.eba.gov.tr/soru-bankasi-kazanim/tde?s=8&d=60&u=0&k=0
Text: 12. Sınıf
URL: https://ogmmateryal.eba.gov.tr/soru-bankasi-kazanim/tde?s=9&d=61&u=0&k=0
Text: 9. Sınıf (2017-23 Müfredatı)
URL: https://ogmmateryal.eba.gov.tr/soru-bankasi-kazanim/tde?s=6&d=58&u=0&k=0


In [36]:
# Get first lesson's links
first_lesson_links = lessons[first_lesson]

# Dictionary to store topics for each link
link_topics = {}

# Process each link in first lesson
for link in first_lesson_links:
    # Get the page content
    response = requests.get(link['href'])
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Create topics dictionary for this link
    topics = {}
    
    # Find all unit items
    unit_items = soup.find_all('li', class_='units-Select-item')
    
    for item in unit_items:
        # Find the bold header (topic name)
        topic_header = item.find(class_='fw-bold')
        if topic_header:
            topic_name = topic_header.get_text(strip=True)
            # Get all text content after removing the bold header
            topic_header.decompose()
            topic_content = []
            for label in item.find_all('label'):
                input_elem = label.find('input')
                if input_elem and input_elem.get('value'):
                    topic_content.append({
                        'value': input_elem.get('value'),
                        'text': label.get_text(strip=True)
                    })
            
            # Store in topics dictionary for this link
            # If we haven't seen this topic before, create a new empty list for it
            if topic_name not in topics:
                topics[topic_name] = []
            # Add this topic's content to the list of contents for this topic
            # The append() method adds an item to the end of a list
            topics[topic_name] = topic_content
    
    # Store topics dictionary for this link
    link_topics[link['href']] = topics

print("\nExtracted topics by link:")
for link_url, topics in link_topics.items():
    print(f"\nLink: {link_url}")
    for topic, contents in topics.items():
        print(f"\n{topic}:")
        for content in contents:
            print(f"- {content}")


          





Extracted topics by link:

Link: https://ogmmateryal.eba.gov.tr/soru-bankasi-kazanim/tde?s=7&d=59&u=0&k=0

Giriş:
- {'value': '0', 'text': ''}
- {'value': '1455', 'text': '10.1.- Giriş'}

Hikâye:
- {'value': '0', 'text': ''}
- {'value': '1456', 'text': 'A.2.1.- Metinde geçen kelime ve kelime gruplarının anlamlarını tespit eder.'}
- {'value': '1457', 'text': 'A.2.2.- Metnin türünün ortaya çıkışı ve tarihsel dönem ile ilişkisini belirler.'}
- {'value': '1458', 'text': 'A.2.3.- Metnin tema ve konusunu belirler.'}
- {'value': '1459', 'text': 'A.2.4.- Metindeki çatışmaları belirler.'}
- {'value': '1460', 'text': 'A.2.5.- Metnin olay örgüsünü belirler.'}
- {'value': '1461', 'text': 'A.2.6.- Metindeki şahıs kadrosunun özelliklerini belirler.'}
- {'value': '1462', 'text': 'A.2.7.- Metindeki zaman ve mekânın özelliklerini belirler.'}
- {'value': '1463', 'text': 'A.2.8.- Metinde anlatıcı ve bakış açısının işlevini belirler.'}
- {'value': '1464', 'text': 'A.2.9.- Metindeki anlatım biçimleri ve t

In [100]:
  # Save the data to a JSON file
import json
            
output_file = 'lesson_topics.json'
with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(link_topics, f, ensure_ascii=False, indent=4)
                
        print(f"\nData saved to {output_file}")


Data saved to lesson_topics.json


In [111]:
# Read and parse the JSON file
with open('lesson_topics.json', 'r', encoding='utf-8') as f:
    lesson_data = json.load(f)

# Iterate through each URL (key) and its topics (values)


topicsLinks = []

for url, topics_map in list(lesson_data.items())[:1]:
    lesson_name = url.split('/')[-1]
    topicIds = []
    for topic_name, topic_contents in list(topics_map.items())[:2]:
        for content in topic_contents:
            if content['value'] != '0' and content['value']:
                topicIds.append(content['value'])
    lesson_name = lesson_name.split('&k=')[0]  # Remove the k parameter from lesson_name
    base_url = f"https://ogmmateryal.eba.gov.tr/soru-bankasi-soru-tipi-sec/{lesson_name}&k={','.join(topicIds)}&t=css"
    topicsLinks.append(base_url)





In [112]:
import requests
import json

# Initialize list to store questions
topicsQuestions = []

# Extract k parameter values from topicsLinks
for url in topicsLinks:
    print(url)
    # Get k parameter value from URL 
    k_param = url.split('&k=')[1].split('&')[0]
    
    # Make API request
    api_url = f"https://ogmmateryal.eba.gov.tr/api/soru-tur-listele?id={k_param}"
    response = requests.get(api_url)
    
    if response.status_code == 200:
        data = response.json()
        # Extract id and cevap for each question
        for question in data:
            question_dict = {
                'id': question['id'],
                'cevap': question['cevap']
            }
            topicsQuestions.append(question_dict)
    else:
        print(f"Error fetching data for k={k_param}: {response.status_code}")

# Save questions to JSON file
output_file = 'topics_questions.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(topicsQuestions, f, ensure_ascii=False, indent=4)
    
print(f"\nQuestions data saved to {output_file}")


https://ogmmateryal.eba.gov.tr/soru-bankasi-soru-tipi-sec/tde?s=7&d=59&u=0&k=1455,1456,1457,1458,1459,1460,1461,1462,1463,1464,1465,1466,1467,1468,1469,1470,1471,1472,1473,1474,1475,1476,1477,1478,1479,1480,1481,1482,1483,1484,1485,1486,1487,1488,1489,1490,1491,1492,1493,1494,1495,1496,1497,1498,1499,1500,1501,1502,1503,1504,1505,1506,1507,1508&t=css

Questions data saved to topics_questions.json


In [115]:
import requests
import json


questionsUrl = ""

# Load questions from topics_questions.json
with open('topics_questions.json', 'r', encoding='utf-8') as f:
    questions_data = json.load(f)

# Extract question IDs for questions with cevap < 5
question_ids = [str(q['id']) for q in questions_data if q['cevap'] < 6]

# For each topic link
for url in topicsLinks:
    # Extract parameters from URL
    params = dict(param.split('=') for param in url.split('?')[1].split('&'))
    
    # Build test URL with all question IDs
    all_ids = ','.join(question_ids)
    questionsUrl = f"https://ogmmateryal.eba.gov.tr/soru-bankasi/tde/test"
    questionsUrl += f"?s={params['s']}&d={params['d']}&u={params['u']}"
    questionsUrl += f"&k={params['k']}&id={all_ids}&p=1&t=css&ks=0&os=0&zs=0"
    print(questionsUrl)
    
    response = requests.get(questionsUrl)
    
    if response.status_code == 200:
        print(f"Successfully accessed test URL: {questionsUrl}")
    else:
        print(f"Error accessing test URL: {questionsUrl}")
        print(f"Status code: {response.status_code}")

print("Completed accessing all test URLs")


https://ogmmateryal.eba.gov.tr/soru-bankasi/tde/test?s=7&d=59&u=0&k=1455,1456,1457,1458,1459,1460,1461,1462,1463,1464,1465,1466,1467,1468,1469,1470,1471,1472,1473,1474,1475,1476,1477,1478,1479,1480,1481,1482,1483,1484,1485,1486,1487,1488,1489,1490,1491,1492,1493,1494,1495,1496,1497,1498,1499,1500,1501,1502,1503,1504,1505,1506,1507,1508&id=8284,8925,8926,8927,8928,8929,8930,8931,8932,8933,8934,8935,8936,8937,8938,8939,8940,8941,8942,8943,8944,8945,8946,8947,8948,8949,8950,8951,8952,8953,8954,8955,8956,8957,8958,8959,8960,8962,8964,8965,8971,9007,9008,9010,9011,9012,9014,9015,9016,9017,9018,9020,9023,9024,9026,9027,9028,9029,9032,9033,9258,9259,9274,9278,9281,9285,9286,9288,9289,9341,9342,9343,9344,9345,9346,9347,9348,9349,9352,9355,9356,9357,9361,9363,9516,9578,9643,9940,22402,22404,22405,22406,22407,22408,22409,24594,24598,24605,24615,24634,24663,24693,24709,24719,24726,24731,24761,24768,24773,24804,24812,24818,24840,24847,24853,24862,24868,24873,24879,24900,24925,24929,24942,24947,249

In [116]:
import requests
import json

# Make request to test URL
response = requests.get(questionsUrl)

if response.status_code == 200:
    print(f"Successfully downloaded test data")
    # Save response to file
    with open('test_data.html', 'w', encoding='utf-8') as f:
        f.write(response.text)
    print("Test data saved to test_data.html")
else:
    print(f"Error downloading test data: {response.status_code}")


Successfully downloaded test data
Test data saved to test_data.html
